# Posterior and Marginal distributions

This notebook is the continuation of `bandwidths.ipynb` notebook, apliying the bw results and the class for get the n-dimentional PDF. 

#### Some considerations: 
1. As **prior** information we will take the cleaned (without NaN or inf values) information from exoplanet.eu, this is the first part of notebook.

2. The PDF from `oiptimal_pdf` class  fulfills the functions of **likelihood** for a certain number of variables in synthetic systems with no-pertutbation, low perturbation and high perturbation. 

3. To get the **marginal** distributions of a variable of interest, we go in the same way that the example marginalization in the notebook `3D.ipynb`.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')

In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

## 1. Data Cleaning

### 1.1 Simulation data   

see the notation according to <a href="https://github.com/saint-germain/population_synthesis/blob/master/README.md ">Readme.md</a> from `Population_synthesis` project.





In [3]:
#========================== Simulation Data ==========================
dn=pd.read_csv('data/proc_no_pert.csv',index_col=None); #without pertubtations 
dl=pd.read_csv('data/proc_lo_pert.csv',index_col=None); #with low pertubtations 
dh=pd.read_csv('data/proc_hi_pert.csv',index_col=None); #with high pertubtations

In [4]:
#dh.head()

### 1.2 Observational data 

Data get from <a href="http://exoplanet.eu/">exoplanet.eu</a>

In [5]:
data_o = pd.read_csv('data/exoplanet.eu_catalog_21.csv', 
                       usecols = ['mass','mass_error_min','semi_major_axis','semi_major_axis_error_min','star_name',
                                  'star_metallicity','star_metallicity_error_min','star_mass','star_mass_error_min'])

data_o = data_o.replace([np.inf, -np.inf], np.nan) 
data_o = data_o.replace([0], np.nan)
data_o = data_o.dropna()

In [6]:
#data_o

In [7]:
data_o = data_o[['star_name','mass','mass_error_min', 'semi_major_axis', 'semi_major_axis_error_min', 
                 'star_metallicity', 'star_metallicity_error_min', 'star_mass', 'star_mass_error_min']]

data_o = data_o.assign(CM_i = data_o["semi_major_axis"]*data_o["mass"])
data_o = data_o.assign(dCM_i= data_o["CM_i"]*(data_o["mass_error_min"].divide(data_o["mass"])+data_o["semi_major_axis_error_min"].divide(data_o["semi_major_axis"])))

data_o["M_rock"] = np.where(data_o["mass"]<=0.03149, data_o["mass"]*317.5, 0)
data_o["dM_rock"] = np.where(data_o["mass"]<=0.03149, data_o["mass_error_min"]*317.5,0)

data_o["M_giant"] = np.where(data_o["mass"]>0.03149, data_o["mass"],0)
data_o["dM_giant"] = np.where(data_o["mass"]>0.03149, data_o["mass_error_min"],0)

data_o["N_rock"] = np.where(data_o["mass"]<=0.03149, 1,0)
data_o["N_giant"] = np.where(data_o["mass"]>0.03149, 1,0)

example = data_o[data_o.star_name == "Kepler-289"]
example

,star_name,mass,mass_error_min,semi_major_axis,semi_major_axis_error_min,star_metallicity,star_metallicity_error_min,star_mass,star_mass_error_min,CM_i,dCM_i,M_rock,dM_rock,M_giant,dM_giant,N_rock,N_giant
2927,Kepler-289,0.023,0.021,0.21,0.01,0.05,0.04,1.08,0.02,0.00483,0.00464,7.3025,6.6675,0.00,0.00,1,0
2928,Kepler-289,0.420,0.050,0.51,0.03,0.05,0.04,1.08,0.02,0.21420,0.03810,0.0000,0.0000,0.42,0.05,0,1
2929,Kepler-289,0.013,0.003,0.33,0.02,0.05,0.04,1.08,0.02,0.00429,0.00125,4.1275,0.9525,0.00,0.00,1,0


In [8]:
np.count_nonzero(example["N_rock"])

2

In [9]:
number=(data_o["star_name"].value_counts()).to_frame()

In [10]:
def NewData(data):
   
    p_system = data.groupby("star_name")

    ms    = p_system['star_mass'].mean();       dms    = p_system['star_mass_error_min'].mean()
    metal = p_system['star_metallicity'].mean();dmetal = p_system['star_metallicity_error_min'].mean()
    
    Mtp   = p_system['mass'].sum();             dMtp   = p_system['mass_error_min'].sum(); 
    CoM   = p_system['CM_i'].sum().divide(p_system["mass"].sum())
    dCoM  = CoM*((p_system['dCM_i'].sum()/p_system['CM_i'].sum()) + (p_system['mass_error_min'].sum())/p_system['mass'].sum())
    
    M_r   = p_system['M_rock'].sum();           dM_r   = p_system["dM_rock"].sum()
    M_gi  = p_system['M_giant'].sum();          dM_gi  = p_system["dM_giant"].sum()    

    N_r  = p_system["N_rock"].sum()
    N_gi  = p_system["N_giant"].sum()
    
    NewData = pd.DataFrame({'sys_name':list(p_system.groups.keys()),
                            'ms':ms, 'dms':dms, 'metal':metal, 'dmetal':dmetal,
                            'Mtp':Mtp, 'dMtp':dMtp,
                            'M_r':M_r, 'dM_r':dM_r,
                            'M_gi':M_gi, 'dM_gi':dM_gi,
                            'com':CoM.tolist(), 'dcom':dCoM.tolist(), 
                            'N_r':N_r, 'N_gi':N_gi})
    return NewData

In [11]:
obs_data = NewData(data_o)
obs_data = obs_data.assign(n_planets = number)
obs_data = obs_data.sort_values(by=['n_planets'], ascending=False)

In [12]:
systm = obs_data[obs_data.sys_name == "Kepler-289"]
systm

,sys_name,ms,dms,metal,dmetal,Mtp,dMtp,M_r,dM_r,M_gi,dM_gi,com,dcom,N_r,N_gi,n_planets
star_name,,,,,,,,,,,,,,,,
Kepler-289,Kepler-289,1.08,0.02,0.05,0.04,0.456,0.074,11.43,7.62,0.42,0.05,0.489737,0.175944,2,1,3


In [13]:
#obs_data.to_csv(r"data/observations.csv", index = False)

In [14]:
#system_i = data_o[data_o.star_name == "Kepler-289"]  
#system_i

## 2. Posterior probability 

At first example we take the system "Kepler-289" to get the prior information of stellar mass `ms` and its metallicity `metal`, reported in the catalog of <a href="http://exoplanet.eu/">exoplanet.eu</a>

In [15]:
systm = obs_data[obs_data.sys_name == "Kepler-289"] 
systm

,sys_name,ms,dms,metal,dmetal,Mtp,dMtp,M_r,dM_r,M_gi,dM_gi,com,dcom,N_r,N_gi,n_planets
star_name,,,,,,,,,,,,,,,,
Kepler-289,Kepler-289,1.08,0.02,0.05,0.04,0.456,0.074,11.43,7.62,0.42,0.05,0.489737,0.175944,2,1,3


### 2.1 Likelihood
In our formalism, for a variable of interest $x^i_k$, the likelihood of $x^i_k$ given $\boldsymbol\gamma^i$, comes from the model as the sum over unknown parameters $\boldsymbol\alpha^i_k$,

$$p\left(x^{i}_k \mid \boldsymbol\gamma^{i}\right)= \int p_{\text{model}}\left(x^{i}_k, \boldsymbol\alpha^{i}_k \mid \boldsymbol\gamma^{i}\right) d\boldsymbol\alpha^{i}_k.$$

This is implemented in the notebook `optimal_pdf`.

In [16]:
import nbimporter
from bandwidths import optimal_pdf #import the class for get the pdf.

Importing Jupyter notebook from bandwidths.ipynb


In this example we take the variables `dng.ms`, `dng.metal` and `dng.nplanets` 

In [ ]:
likelihood = optimal_pdf(dn.ms, dn.metal, dn.nplanets)
likelihood.pdf()

In [ ]:
#likelihood.pdf_std

### 2.2 Prior infromation

The prior information $p(\boldsymbol\gamma^{i})$ comes from observations of individual known properties in the pseudovector $\boldsymbol\gamma^{i}$.

To generate a 2D prior $p(\boldsymbol\gamma^{i})$, we multiply a 2D matrix of ones times each prior. Here, we multiply the first prior normaly but for the second prior it is necessary to transpose the previous result. 

$$\text{prior} = ((1_{n\times n}\times Px_n)^{t}\times Py_n)^{t}$$

Finally, we transpose the result to get the prior distribution. For more dimensions the process is the same

In [ ]:
from scipy.stats import truncnorm

In [ ]:
# per data needs a list with [mu, sigma]: 
#var1 nd var 2 are the variables from simulations args are the list [mu, sigma]:
class prior():
    
    def __init__(self, var1, var2, *args):
        self.org_data = np.array(args)
        self.data = np.vstack([var1,var2]).T
        self.data_std = (self.data-np.mean(self.data, axis=0)) / np.std(self.data, axis=0) # standarization       
        
    def prior_pdf(self, lenght = 150):
        self.X, self.pdfs = [], []     
        self.mu, self.sig = [], []
        for i in range(len(self.org_data)):
            
            self.x = np.linspace(self.data_std[:,i].min(), 
                                 self.data_std[:,i].max(), lenght)
            
            mu = (self.org_data[i][0] - np.mean(self.data, axis=0)[i])/np.std(self.data, axis=0)[i]
            sig= self.org_data[i][1]/np.std(prueba.data, axis = 0)[i]
            print(mu,sig)
            
            pdf = stats.norm.pdf(self.x,loc = mu, scale = sig)       
            self.X.append(self.x)
            self.pdfs.append(pdf)
            self.mu.append(mu); self.sig.append(sig)
            
        if len(self.org_data) == 2:
            M_ones = np.ones([lenght, lenght]) 
            prior = ((M_ones*self.pdfs[0]).T*self.pdfs[1]).T     
            
            self.pdf_prior = prior

In [ ]:
prueba = prior(dn.ms, dn.metal, 
               [systm.ms, systm.dms], 
               [systm.metal,systm.dmetal])

prueba.prior_pdf()

In [ ]:
plt.contour(prueba.X[0], prueba.X[1], prueba.pdf_prior)
plt.plot(prueba.X[0], prueba.pdfs[0])
plt.plot(prueba.pdfs[1],prueba.X[1])

In [ ]:
from scipy.stats import multivariate_normal
mean = [prueba.mu[0][0], prueba.mu[1][0]]
cov = [[prueba.sig[0][0]**2,0],[0, prueba.sig[1][0]**2]]


x, y = np.meshgrid(prueba.X[0], prueba.X[1])
#x, y = np.mgrid[prueba.X[0].min():prueba.X[0].max(),prueba.X[1].min():prueba.X[1].max()]
rv = multivariate_normal(mean, cov)

n_pdf = rv.pdf(np.dstack((x, y)))

fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ff = ax2.contour(x, y, rv.pdf(np.dstack((x, y))))
cbar = fig2.colorbar(ff)

plt.plot(prueba.X[0], prueba.pdfs[0])
plt.plot(prueba.pdfs[1],prueba.X[1])

In [ ]:
n_pdf

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,4))
fig.subplots_adjust(wspace=.3)
lat = 13
#1
ax[0].plot(prueba.X[0],prueba.pdfs[0],label = "ms"+str(prueba.mu[0]))
#ax[0].plot(likelihood.std_interval[0], prueba.pdfs[0], label = "prior stellar mass")
ax[0].set_xlabel("stellar mass", fontsize =lat)
ax[0].legend(fontsize =lat)

#2
#ax[1].plot(prueba.pdfs[1], likelihood.real_interval[1], label = "prior metallicity")
ax[1].plot(prueba.pdfs[1], prueba.X[1], label = "metallicity"+str(prueba.mu[1]))
ax[1].plot(prueba.pdfs[0], prueba.X[0], label = "metallicity"+str(prueba.mu[1]))

ax[1].legend(fontsize =lat)
#3
ax[2].contourf(prueba.X[0], prueba.X[1], prueba.pdf_prior)
ax[2].set_xlabel("stellar mass", fontsize =lat)
ax[2].set_ylabel("metallicity", fontsize =lat)


#### prior plot

Plots for the prior distribution which comes from observations

In [ ]:
from matplotlib import rc, cm, rcParams
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
rc('text', usetex=True)
rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

In [ ]:
#x an y needs to be standarized variables 
#z is the pdf
def prior_2d(x, xy, y, yy, z, name):

    lat = 12
    fig, axs = plt.subplots(1, 1, figsize = (7.5,5), constrained_layout=True)
    gs = gridspec.GridSpec(3, 4)
    ax_main = plt.subplot(gs[1:, 1:3])
    
    plot = ax_main.contourf(x, y, z, 10, cmap=plt.cm.viridis)
    
    ax_main.set_xlabel(r"Standarized setellar mass $M_{\star}^{\prime}$", fontsize = lat+1)
    
    #ax_main.tick_params(labelsize=lat-1)  
    
    ax_main.set_yticklabels([])
    ax_main.tick_params(axis = "y", labelsize=0, length = 0)
    
    axins = inset_axes(ax_main, width="5%", # width = 10% of parent_bbox width
                       height="100%", # height : 50%
                       loc=6, bbox_to_anchor=(1.05, 0., 1, 1), 
                       bbox_transform=ax_main.transAxes, borderpad=0)
    
    cbar = plt.colorbar(plot, cax=axins)
    cbar.set_ticks([])
    cbar.set_label(fontsize = lat, label = r'$p(M_{\star}^{\prime}, \mu^{\prime})$')
    
    #ax_main.legend(loc=1, bbox_to_anchor=[0.95,0.95])
    gs.update(wspace=0.1, hspace=0.1) #spaces

    ax_main.set_xticks([-1,-.5,0,.5,1,1.5])
    ax_main.set_xticklabels(["-1.0","-0.5","0","0.5","1.0","1.5"])

    #---------------------------------Marginal-------------------------------------
    
    ax_xDist = plt.subplot(gs[0, 1:3])#,  sharex=ax_main)  #Xdistribution with shared axis
    ax_yDist = plt.subplot(gs[1:3, 0:1])#,sharey=ax_main) #Ydistribution with shared axis
    
    ax_xDist.plot(x,xy/xy.max(),color='C0', label = r"$p\left(M_{\star}^{\prime}\right)$")
    
    ax_xDist.set_xticklabels([]); #ax_xDist.set_yticks([]); ax_yDist.set_xticks([]); 
    ax_xDist.tick_params(axis = "x", which = "both", bottom = False, top = False)
    
    ax_yDist.set_ylabel(r"Satandarized metallycity $\mu^{\prime}$", fontsize = lat+1)
    ax_yDist.plot(yy/yy.max(), y, color='C4', label = r"$p(\mu^{\prime})$")
    ax_yDist.invert_xaxis()
    #ax_yDist.invert_xaxis()
    ax_yDist.legend(fontsize=lat, loc=2); ax_xDist.legend(fontsize=lat)

    plt.savefig("images/plots/"+name+".pdf", bbox_inches='tight')
    plt.show()
    

In [ ]:
prior_2d(prueba.X[0], prueba.pdfs[0], prueba.X[1], prueba.pdfs[1], prueba.pdf_prior,"prior_kepler")         

### 2.3 Posterior and marginal distributions

#### Posterior

The posterior distribution comes form Bayes Theorem, then in our formalims:

$$p^\prime\left(x^{i}_k \mid \boldsymbol\gamma^{i}\right) \propto p\left(x^{i}_k \mid \boldsymbol\gamma^{i}\right) p(\boldsymbol\gamma^{i})$$

In therms of the length of data 

$$\boxed{\text{posterior}_{lmn} \propto \text{likelihood}_{lmn} \times \text{prior}_{mn}}$$

Sum the posterior over $l$ is the `axis=0`, $n$ means the `axis=1` and $n$ is the `axis=2`. Finally if we normalize the posterior the above equation becomes into equality.  

So, the posterior is the product of a likelihood with shape $l=100$, $m=100$ and $n=100$ times the prior with shape $m=100$ and $n=100$: 

In [ ]:
# likelihood is an object from "class optimal_pdf"
# prior is an object from class "prior"

def posterior(likelihood,prior):
    
    diff = [np.abs(likelihood.data_std[:,i][1] - likelihood.data_std[:,i][0]) for i in range(len(likelihood.data_std[0]))] 

    post = (likelihood.pdf_std.T*prior.T).T
    
    
    if likelihood.dim == 3:
        norm = ((post.sum(axis=0)*(diff[0])).sum(axis=0)*(diff[1])).sum()*(diff[2])
    
    elif likelihood.dim == 4:
        norm = (((post.sum(axis=0)*(diff[0])).sum(axis=0)*(diff[1])).sum(axis=0)*(diff[2])).sum()*diff[3]
    
    #print((((post/norm)*diff[0]).sum(axis=0)*diff[1]).sum()*diff[2])
    
    return post/norm

In [ ]:
#pr = prior(dn.ms, dn.metal, [systm.ms, systm.dms], [systm.metal,systm.dmetal])
#pr.prior_pdf()

pr = n_pdf

In [ ]:
post = posterior(likelihood, pr)

In [ ]:
#import plotly.graph_objects as go
#from plotly.subplots import make_subplots

#fig = go.Figure(data=go.Volume(
#    x=likelihood.std_grid[0].flatten(),
#    y=likelihood.std_grid[0].flatten(),
#    z=likelihood.std_grid[0].flatten(),
#    value=post.flatten(),
#    opacity=0.15,     # needs to be small to see through all surfaces
#    surface_count=20, # needs to be a large number for good volume rendering
#    ))
#fig.write_image("images/fig1.pdf")

#### Marginal distriution 

The parginal distribution for a variable of interest is, 

$$p\left(x^{i}_{k}\right)=\int p\left(x^{i}_{k} \mid \boldsymbol\gamma^{i}\right) p(\boldsymbol\gamma^{i})\ d\boldsymbol\gamma^{i}.$$

in other words, 

$$p\left(x^{i}_{k}\right)=\sum_{j=1}^{n} p\left(x^{i}_{k_j} \mid \boldsymbol\gamma^{i}_j\right) p\left(\boldsymbol\gamma^{i}_j\right)\ \Delta \boldsymbol\gamma^{i}_j.$$



In our example, we take stellar mass, metallicity and number of planets. The likelihood came from giant planets obervations without perturbations in its disk structure. 

We will get the marginal distribution $p(x^i_k)$ for our variable of interest $x^{i}_k$ which is the number of planets in the system "Kepler-289".







In [ ]:
def marginal(posterior, likelihood):
    
    diff = [np.abs(likelihood.data_std[:,i][1] - likelihood.data_std[:,i][0]) for i in range(len(likelihood.data_std[0]))] 
    #print(diff)
    M  = (posterior.sum(axis=0)*diff[0]).sum(axis=0)*diff[1]
    
    dz = np.abs(likelihood.real_interval[2][1]-likelihood.real_interval[2][0])
    #dz=diff[2] 
    
    norm = (M*dz).sum()
    marginal = M/norm # norm in the real space to work the variable in it space
    
    #print((marginal*dz).sum())
    
    return marginal

In [ ]:
Np_marginal = marginal(post, likelihood)

In [ ]:
dz = likelihood.real_interval[2][1]-likelihood.real_interval[2][0]
mean = (Np_marginal*likelihood.real_interval[2]*(dz)).sum()

In [ ]:
std = ((Np_marginal)*((likelihood.real_interval[2]-mean)**2)*dz).sum()**(1/2.)

In [ ]:
inte = np.cumsum(Np_marginal)*dz

p_25 = likelihood.real_interval[2][np.argmin((inte-0.25)**2)]
p_50 = likelihood.real_interval[2][np.argmin((inte-0.50)**2)]
p_75 = likelihood.real_interval[2][np.argmin((inte-0.75)**2)]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,4))

ax[0].plot(likelihood.real_interval[2], Np_marginal/Np_marginal.max(), label="p(nplanets)")
ax[0].set_xlabel("nplanets")
ax[0].set_ylabel("distribution")
ax[0].axvline(x = systm.n_planets.values, ls='--', c="k", label = "Observed planets = 3")
ax[0].axvline(x = mean, ls='--', c="C1", label = r"$\mu = 11$")
ax[0].axvline(x = mean+std, ls=':', c="C1", label = r"$\sigma$")
ax[0].axvline(x = mean+2*std, ls=':', c="C2", label = r"$2\sigma$")
ax[0].axvline(x = mean-2*std, ls=':', c="C2")
ax[0].legend()


ax[1].plot(likelihood.real_interval[2], Np_marginal/Np_marginal.max(), label="p(nplanets)")
ax[1].set_xlabel("nplanets")
ax[1].set_ylabel("distribution")

ax[1].axvline(x = systm.n_planets.values, ls='--', c="k", label = "Observed planets = 3")
ax[1].axvline(x = p_25, ls='--', c="C1", label = r"$25\%$")
ax[1].axvline(x = p_50, ls='--', c="C2", label = r"$50\%$")
ax[1].axvline(x = p_75, ls='--', c="C3", label = r"$75\%$")

ax[1].legend()

#### Posterior and marginal distributions - the class and its methods

In [ ]:
#like and prior are pdfs from prior class and optimal_pdf class.
#args are the variables in the same order than likelihood.

class Marginal():
    
    def __init__(self, like, prior, *args):
                
        self.like = like; self.prior= prior 
        self.space = [np.linspace(args[i].min(),args[i].max(), 
                                  150) for i in range(len(args))]
    
        self.data = np.vstack([*args]).T
        self.data_std = (self.data-np.mean(self.data, axis=0))/np.std(self.data, axis=0) # standarization
        
        self.diff  = [np.abs(self.data_std[:,i][1]-self.data_std[:,i][0]) for i in range(len(self.data_std[0]))] 
        self.dz = np.abs(self.space[2][1]-self.space[2][0])
        self.z = self.space[2]
        
    def posterior(self):
        post = (self.like.T*self.prior.T).T
        norm = ((post.sum(axis=0)*(self.diff[0])).sum(axis=0)*(self.diff[1])).sum()*(self.diff[2])
        self.post = post/norm
    
    def pdf(self):
        self.posterior()
        M  = (self.post.sum(axis=0)*self.diff[0]).sum(axis=0)*self.diff[1]
        norm = (M*self.dz).sum()
        self.marginal = M/norm

        inte = np.cumsum(self.marginal)*self.dz

        self.z_mean = (self.marginal*self.z*self.dz).sum()
        self.z_std  = ((self.marginal)*((self.z-self.z_mean)**2)*self.dz).sum()**(1/2.)
        
        self.p_25 = self.z[np.argmin((inte-0.25)**2)]
        self.p_50 = self.z[np.argmin((inte-0.50)**2)]
        self.p_75 = self.z[np.argmin((inte-0.75)**2)]
        
        #return self.marginal

In [ ]:
def m_plot(marginal, name, i = False, sci = False, lims = []):
    
    like_z = marginal.space[2]
    lat = 15
    plt.plot(like_z, marginal.marginal/marginal.marginal.max(), 
             label = r"Distribution")
    if i == True: 
        plt.axvline(x = round(marginal.p_25), ls='--', c="C1",
                    label = "25\% = " + str(int(round(marginal.p_25))))
        plt.axvline(x = round(marginal.p_50), ls='--', c="C2", 
                    label = "50\% = " + str(int(round(marginal.p_50))))
        plt.axvline(x = round(marginal.p_75), ls='--', c="C3", 
                    label = "75\% = " + str(int(round(marginal.p_75))))
    elif sci ==True:    
        plt.axvline(x = marginal.p_25, ls='--', c="C1", 
                    label = "25\% = " + "{:.2e}".format(marginal.p_25))
        plt.axvline(x = marginal.p_50, ls='--', c="C2", 
                    label = "50\% = " + "{:.2e}".format(marginal.p_50))
        plt.axvline(x = marginal.p_75, ls='--', c="C3", 
                    label = "75\% = " + "{:.2e}".format(marginal.p_75))
    
    elif lims:
        plt.xlim(lims[0], lims[1])
        plt.axvline(x = marginal.p_25, ls='--', c="C1", 
                    label = "25\% = " + "%.3f"%marginal.p_25)
        plt.axvline(x = marginal.p_50, ls='--', c="C2", 
                    label = "50\% = " + "%.3f"%marginal.p_50)
        plt.axvline(x = marginal.p_75, ls='--', c="C3", 
                    label = "75\% = " + "%.3f"%marginal.p_75)
    
    else: 
        plt.axvline(x = marginal.p_25, ls='--', c="C1", 
                    label = "25\% = " + "%.3f"%marginal.p_25)
        plt.axvline(x = marginal.p_50, ls='--', c="C2", 
                    label = "50\% = " + "%.3f"%marginal.p_50)
        plt.axvline(x = marginal.p_75, ls='--', c="C3", 
                    label = "75\% = " + "%.3f"%marginal.p_75)
    
    #plt.axhline(0.25, lw=.5, ls=":")
    #plt.axhline(0.5, lw=.5, ls=":")
    #plt.axhline(0.75, lw=.5, ls=":")
    #plt.plot(like_z, np.cumsum(marginal.marginal)*marginal.dz)
    #plt.xticks(fontsize= lat-2); plt.yticks(fontsize= lat-2)

    plt.xlabel(str(name), fontsize=lat+1)
    
    plt.tick_params(axis='both', which='minor', labelsize=lat-1)
    plt.legend(fontsize=lat-1)

# Marginal plots

Below are marginal plots of the selected systems.
The simulation information

In [ ]:
dn = pd.read_csv('data/no_p.csv',index_col=None)
dl = pd.read_csv('data/low_p.csv',index_col=None)
dh = pd.read_csv('data/high_p.csv',index_col=None)

In [ ]:
dn.head()

Now we reshape the likelihood pdf for each variable:

In [ ]:
Md  = pd.read_csv('data/likelihoods/like_md.csv',index_col=None)
#tau = pd.read_csv('data/likelihoods/like_tgas.csv',index_col=None)
#com = pd.read_csv('data/likelihoods/like_com.csv',index_col=None)
#mtp = pd.read_csv('data/likelihoods/like_Mtp.csv',index_col=None)
#mjup= pd.read_csv('data/likelihoods/like_Mjup.csv',index_col=None)
#mrock = pd.read_csv('data/likelihoods/like_Mrock.csv',index_col=None)
#ngi = pd.read_csv('data/likelihoods/like_ngi.csv',index_col=None)
#ntp = pd.read_csv('data/likelihoods/like_ntp.csv',index_col=None)
#nplanets = pd.read_csv('data/likelihoods/like_nplanets.csv',index_col=None)

In [ ]:
# 1 to 4 because of the number of columns in DataFrames 
like_Md  = [Md[str(Md.columns[i])].values.reshape(150,150,150) for i in range(1,4)]
#like_tau = [tau[str(tau.columns[i])].values.reshape(150,150,150) for i in range(1,4)]
#like_com = [com[str(com.columns[i])].values.reshape(150,150,150) for i in range(1,4)]
#like_mtp = [mtp[str(mtp.columns[i])].values.reshape(150,150,150) for i in range(1,4)]
#like_mjup = [mjup[str(mjup.columns[i])].values.reshape(150,150,150) for i in range(1,4)]
#like_mrock = [mrock[str(mrock.columns[i])].values.reshape(150,150,150) for i in range(1,4)]
#like_ngi = [ngi[str(ngi.columns[i])].values.reshape(150,150,150) for i in range(1,4)]
#like_ntp = [ntp[str(ntp.columns[i])].values.reshape(150,150,150) for i in range(1,4)]
#like_nplanets = [nplanets[str(nplanets.columns[i])].values.reshape(150,150,150) for i in range(1,4)]

likelihoods = [like_Md]#, like_tau, like_com, like_mtp, like_mjup, 
               #like_mrock, like_ngi, like_ntp, like_nplanets]

## Kepler-289 

this is just and example of  number of plants

In [ ]:
# System and Prior:
systm = obs_data[obs_data.sys_name == "Kepler-289"] 

prior_keplern = prior(dn.ms, dn.metal, 
                     [systm.ms, systm.dms], 
                     [systm.metal,systm.dmetal])

prior_keplern.prior_pdf()
prior_keplern = prior_keplern.pdf_prior

prior_keplerl = prior(dl.ms, dl.metal, 
                     [systm.ms, systm.dms], 
                     [systm.metal,systm.dmetal])

prior_keplerl.prior_pdf()
prior_keplerl = prior_keplerl.pdf_prior

prior_keplerh = prior(dh.ms, dh.metal, 
                     [systm.ms, systm.dms], 
                     [systm.metal,systm.dmetal])

prior_keplerh.prior_pdf()
prior_keplerh = prior_keplerh.pdf_prior

In [ ]:
liken = like_mrock[0]

In [ ]:
likel = like_mrock[1]

In [ ]:
likeh = like_mrock[2]

In [ ]:
m_nplanetsn = Marginal(liken, prior_keplern, dn.ms, dn.metal, dn.Mrock)
m_nplanetsn.pdf()

m_nplanetsl = Marginal(likel, prior_keplerl, dl.ms, dl.metal, dl.Mrock)
m_nplanetsl.pdf()

m_nplanetsh = Marginal(likeh, prior_keplerh, dh.ms, dh.metal, dh.Mrock)
m_nplanetsh.pdf()

In [ ]:
fig, ax = plt.subplots(sharey='row', figsize=(16,4))

plt.subplot(1,3,1)
m_plot(m_nplanetsn, r"mrock$_n$", lims = [-45, 1500])

plt.subplot(1,3,2)
plt.title("Low perturbations")
m_plot(m_nplanetsl, r"mrock$_l$", lims = [-45, 1500])
plt.yticks([])

plt.subplot(1,3,3)
plt.title("high perturbations")
m_plot(m_nplanetsh, r"mrock$_h$", lims = [-45, 1500])
plt.yticks([])

In [ ]:
dn.head()

In [ ]:
mar = Marginal(like_ngi[0], prior_keplern, dn.ms, dn.metal, dn.ngi)

In [ ]:
mar.pdf()

In [ ]:
plt.plot(mar.space[2],mar.marginal)
plt.xlim(-0.005,4)

The above plots only show the marginal behavior for the number of planets according to the type of perturbation. But we wan to observe the behavior for other variables. 


### For all variables

In [ ]:
import matplotlib as mpl

mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}', r'\usepackage{wasysym}']

In [ ]:
data = [dn, dl, dh]

likelihoods = [like_Md, like_tau, like_com, like_mtp, like_mjup, 
               like_mrock, like_nplanets, like_ngi, like_ntp]

In [ ]:
#priors:
prior_kepler = []

for i in range(len(data)):
    
    kepler = prior(data[i]["ms"], data[i]["metal"], 
                     [systm.ms, systm.dms], 
                     [systm.metal,systm.dmetal])
    kepler.prior_pdf()
    
    prior_kepler.append(kepler.pdf_prior)

In [ ]:
marginals = []
variables = ["md","taugas","com","Mtp","Mjup","Mrock","nplanets","ngi","npt"]

for n, var in enumerate(likelihoods):
    
    M = []
    
    for  i in range(len(data)):
        marginal = Marginal(var[i], prior_kepler[i], data[i]["ms"], 
                            data[i]["metal"], data[i][variables[n]])
        marginal.pdf()
        M.append(marginal)
        
    marginals.append(M)

In [ ]:
def m3_plot(marginals, name, yname):
    
    lat=15
    fig, ax = plt.subplots(sharey='row', figsize=(13,4))
    titles = ["No perturbations", "Low perturbations", "High perturbations"]
    
    for i in range(0,3):
        
        plt.subplot(1,3,i+1)
        m_plot(marginals[i], name)
        plt.title(titles[i], fontsize = lat)
                    
    plt.subplots_adjust(hspace=0.35)
    fig.tight_layout()

In [ ]:
m3_plot(marginals[0], r"Mass of Disk $M_d$ ($M_\odot$)", r"$p\left(M_d\right)$")

In [ ]:
m3_plot(marginals[1], r"Dissipation time $\tau_g$ (y)", r"$p\left(\tau_g\right)$")

In [ ]:
m3_plot(marginals[2], r"Center of mass $r_{\text{cm}}$ (AU)", 
        r"$p\left(r_\text{cm}\right)$")

In [ ]:
m3_plot(marginals[3], r"Total planetary mass $M_{tp}$ ($M_\odot$)", 
        r"$p\left(M_{tp}\right)$")

In [ ]:
m3_plot(marginals[4], r"Gian planetary mass $M_{\jupiter}$ ($M_\text{jup}$)", 
        r"$p\left(M_{\jupiter}\right)$")

In [ ]:
m3_plot(marginals[5], r"Rocky planetary mass $M_{r}$ ($M_{\oplus}$)", 
        r"$p\left(M_{r}\right)$")

In [ ]:
m3_plot(marginals[6], r"Number of total planets $N_{t}$", 
        r"$p\left(N_{t}\right)$")

In [ ]:
m3_plot(marginals[7], r"Number of giants $N_{\jupiter}$", 
        r"$p\left(N_{\jupiter}\right)$")

In [ ]:
m3_plot(marginals[8], r"Number of giants $N_{t}$", 
        r"$p\left(N_{\oplus}\right)$")

 ### The multiplot 